In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle

pd.set_option('max_columns', 90)
pd.set_option('max_rows', 90)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

## 1. Import the data

In [4]:
test_data=pd.read_csv("test.csv")
train_data=pd.read_csv("train.csv")

In [5]:
train_data.head()


,UID,BLOCKID,SUMLEVEL,COUNTYID,STATEID,state,state_ab,city,place,type,primary,zip_code,area_code,lat,lng,ALand,AWater,pop,male_pop,female_pop,rent_mean,rent_median,rent_stdev,rent_sample_weight,rent_samples,rent_gt_10,rent_gt_15,rent_gt_20,rent_gt_25,rent_gt_30,rent_gt_35,rent_gt_40,rent_gt_50,universe_samples,used_samples,hi_mean,hi_median,hi_stdev,hi_sample_weight,hi_samples,family_mean,family_median,family_stdev,family_sample_weight,family_samples,hc_mortgage_mean,hc_mortgage_median,hc_mortgage_stdev,hc_mortgage_sample_weight,hc_mortgage_samples,hc_mean,hc_median,hc_stdev,hc_samples,hc_sample_weight,home_equity_second_mortgage,second_mortgage,home_equity,debt,second_mortgage_cdf,home_equity_cdf,debt_cdf,hs_degree,hs_degree_male,hs_degree_female,male_age_mean,male_age_median,male_age_stdev,male_age_sample_weight,male_age_samples,female_age_mean,female_age_median,female_age_stdev,female_age_sample_weight,female_age_samples,pct_own,married,married_snp,separated,divorced
0,267822,NaN,140,53,36,New York,NY,Hamilton,Hamilton,City,tract,13346,315,42.840812,-75.501524,202183361.0,1699120,5230,2612,2618,769.38638,784.0,232.63967,272.34441,362.0,0.86761,0.79155,0.59155,0.45634,0.42817,0.18592,0.15493,0.12958,387,355,63125.28406,48120.0,49042.01206,1290.96240,2024.0,67994.14790,53245.0,47667.30119,884.33516,1491.0,1414.80295,1223.0,641.22898,377.83135,867.0,570.01530,558.0,270.11299,770.0,499.29293,0.01588,0.02077,0.08919,0.52963,0.43658,0.49087,0.73341,0.89288,0.85880,0.92434,42.48574,44.00000,22.97306,696.42136,2612.0,44.48629,45.33333,22.51276,685.33845,2618.0,0.79046,0.57851,0.01882,0.01240,0.08770
1,246444,NaN,140,141,18,Indiana,IN,South Bend,Roseland,City,tract,46616,574,41.701441,-86.266614,1560828.0,100363,2633,1349,1284,804.87924,848.0,253.46747,312.58622,513.0,0.97410,0.93227,0.69920,0.69920,0.55179,0.41235,0.39044,0.27888,542,502,41931.92593,35186.0,31639.50203,838.74664,1127.0,50670.10337,43023.0,34715.57548,375.28798,554.0,864.41390,784.0,482.27020,316.88320,356.0,351.98293,336.0,125.40457,229.0,189.60606,0.02222,0.02222,0.04274,0.60855,0.42174,0.70823,0.58120,0.90487,0.86947,0.94187,34.84728,32.00000,20.37452,323.90204,1349.0,36.48391,37.58333,23.43353,267.23367,1284.0,0.52483,0.34886,0.01426,0.01426,0.09030
2,245683,NaN,140,63,18,Indiana,IN,Danville,Danville,City,tract,46122,317,39.792202,-86.515246,69561595.0,284193,6881,3643,3238,742.77365,703.0,323.39011,291.85520,378.0,0.95238,0.88624,0.79630,0.66667,0.39153,0.39153,0.28307,0.15873,459,378,84942.68317,74964.0,56811.62186,1155.20980,2488.0,95262.51431,85395.0,49292.67664,709.74925,1889.0,1506.06758,1361.0,731.89394,699.41354,1491.0,556.45986,532.0,184.42175,538.0,323.35354,0.00000,0.00000,0.09512,0.73484,1.00000,0.46332,0.28704,0.94288,0.94616,0.93952,39.38154,40.83333,22.89769,888.29730,3643.0,42.15810,42.83333,23.94119,707.01963,3238.0,0.85331,0.64745,0.02830,0.01607,0.10657
3,279653,NaN,140,127,72,Puerto Rico,PR,San Juan,Guaynabo,Urban,tract,927,787,18.396103,-66.104169,1105793.0,0,2700,1141,1559,803.42018,782.0,297.39258,259.30316,368.0,0.94693,0.87151,0.69832,0.61732,0.51397,0.46927,0.35754,0.32961,438,358,48733.67116,37845.0,45100.54010,928.32193,1267.0,56401.68133,44399.0,41082.90515,490.18479,729.0,1175.28642,1101.0,428.98751,261.28471,437.0,288.04047,247.0,185.55887,392.0,314.90566,0.01086,0.01086,0.01086,0.52714,0.53057,0.82530,0.73727,0.91500,0.90755,0.92043,48.64749,48.91667,23.05968,274.98956,1141.0,47.77526,50.58333,24.32015,362.20193,1559.0,0.65037,0.47257,0.02021,0.02021,0.10106
4,247218,NaN,140,161,20,Kansas,KS,Manhattan,Manhattan City,City,tract,66502,785,39.195573,-96.569366,2554403.0,0,5637,2586,3051,938.56493,881.0,392.44096,1005.42886,1704.0,0.99286,0.98247,0.91688,0.84740,0.78247,0.60974,0.55455,0.44416,1725,1540,31834.15466,22497.0,34046.50907,1548.67477,1983.0,54053.42396,50272.0,39609.12605,244.08903,395.0,1192.58759,1125.0,327.49674,76.61052,134.0,443.68855,444.0,76.12674,124.0,79.55556,0.05426,0.05426,0.05426,0.51938,0.18332,0.65545,0.7496

In [6]:
test_data.head()

,UID,BLOCKID,SUMLEVEL,COUNTYID,STATEID,state,state_ab,city,place,type,primary,zip_code,area_code,lat,lng,ALand,AWater,pop,male_pop,female_pop,rent_mean,rent_median,rent_stdev,rent_sample_weight,rent_samples,rent_gt_10,rent_gt_15,rent_gt_20,rent_gt_25,rent_gt_30,rent_gt_35,rent_gt_40,rent_gt_50,universe_samples,used_samples,hi_mean,hi_median,hi_stdev,hi_sample_weight,hi_samples,family_mean,family_median,family_stdev,family_sample_weight,family_samples,hc_mortgage_mean,hc_mortgage_median,hc_mortgage_stdev,hc_mortgage_sample_weight,hc_mortgage_samples,hc_mean,hc_median,hc_stdev,hc_samples,hc_sample_weight,home_equity_second_mortgage,second_mortgage,home_equity,debt,second_mortgage_cdf,home_equity_cdf,debt_cdf,hs_degree,hs_degree_male,hs_degree_female,male_age_mean,male_age_median,male_age_stdev,male_age_sample_weight,male_age_samples,female_age_mean,female_age_median,female_age_stdev,female_age_sample_weight,female_age_samples,pct_own,married,married_snp,separated,divorced
0,255504,NaN,140,163,26,Michigan,MI,Detroit,Dearborn Heights City,CDP,tract,48239,313,42.346422,-83.252823,2711280,39555,3417,1479,1938,858.57169,859.0,232.39082,276.07497,424.0,1.00000,0.95696,0.85316,0.85316,0.85316,0.85316,0.76962,0.63544,435,395,48899.52121,38746.0,44392.20902,798.02401,1180.0,53802.87122,45167.0,43756.56479,464.30972,769.0,1139.24548,1109.0,336.47710,262.67011,474.0,488.51323,436.0,192.75147,271.0,189.18182,0.06443,0.06443,0.07651,0.63624,0.14111,0.55087,0.51965,0.91047,0.92010,0.90391,33.37131,27.83333,22.36768,334.30978,1479.0,34.78682,33.75000,21.58531,416.48097,1938.0,0.70252,0.28217,0.05910,0.03813,0.14299
1,252676,NaN,140,1,23,Maine,ME,Auburn,Auburn City,City,tract,4210,207,44.100724,-70.257832,14778785,2705204,3796,1846,1950,832.68625,750.0,267.22342,183.32299,245.0,1.00000,1.00000,0.86611,0.67364,0.30962,0.30962,0.30962,0.27197,275,239,72335.33234,61008.0,51895.81159,922.82969,1722.0,85642.22095,74759.0,49156.72870,482.99945,1147.0,1533.25988,1438.0,536.61118,373.96188,937.0,661.31296,668.0,201.31365,510.0,279.69697,0.01175,0.01175,0.14375,0.64755,0.52310,0.26442,0.49359,0.94290,0.92832,0.95736,43.88680,46.08333,22.90302,427.10824,1846.0,44.23451,46.66667,22.37036,532.03505,1950.0,0.85128,0.64221,0.02338,0.00000,0.13377
2,276314,NaN,140,15,42,Pennsylvania,PA,Pine City,Millerton,Borough,tract,14871,607,41.948556,-76.783808,258903666,863840,3944,2065,1879,816.00639,755.0,416.25699,141.39063,217.0,0.97573,0.93204,0.78641,0.71845,0.63592,0.47573,0.43689,0.32524,245,206,58501.15901,51648.0,45245.27248,893.07759,1461.0,65694.06582,57186.0,44239.31893,619.73962,1084.0,1254.54462,1089.0,596.85204,340.45884,552.0,397.44466,356.0,189.40372,664.0,534.16737,0.01069,0.01316,0.06497,0.45395,0.51066,0.60484,0.83848,0.89238,0.86003,0.92463,39.81661,41.91667,24.29111,499.10080,2065.0,41.62426,44.50000,22.86213,453.11959,1879.0,0.81897,0.59961,0.01746,0.01358,0.10026
3,248614,NaN,140,231,21,Kentucky,KY,Monticello,Monticello City,City,tract,42633,606,36.746009,-84.766870,501694825,2623067,2508,1427,1081,418.68937,385.0,156.92024,88.95960,93.0,1.00000,0.93548,0.93548,0.64516,0.55914,0.46237,0.46237,0.36559,153,93,38237.55059,31612.0,34527.61607,775.17947,957.0,44156.38709,34687.0,34899.74300,535.21987,689.0,862.65763,749.0,624.42157,299.56752,337.0,200.88113,180.0,91.56490,467.0,454.85404,0.00995,0.00995,0.01741,0.41915,0.53770,0.80931,0.87403,0.60908,0.56584,0.65947,41.81638,43.00000,24.65325,333.57733,1427.0,44.81200,48.00000,21.03155,263.94320,1081.0,0.84609,0.56953,0.05492,0.04694,0.12489
4,286865,NaN,140,355,48,Texas,TX,Corpus Christi,Edroy,Town,tract,78410,361,27.882461,-97.678586,13796057,497689,6230,3274,2956,1031.63763,997.0,326.76727,277.39844,624.0,0.72276,0.66506,0.53526,0.38301,0.18910,0.16667,0.14263,0.11058,660,624,114456.07790,94211.0,81950.95692,836.30759,2404.0,123527.02420,103898.0,72173.55823,507.42257,1738.0,1996.41425,1907.0,740.21168,319.97570,1102.0,867.57713,804.0,376.20236,642.0,333.91919,0.00000,0.00000,0.03440,0.63188,1.000

## 2. Find the primary key

UID is the primary key, as it is unique reference key.

## 3.Gauge the fill rate of the variables and devise plans for missing value treatment. Please explain explicitly the reason for the treatment chosen for each variable. 

First lets perform operations on the training data, then perform on the test data.

In [7]:
train_data.dtypes

UID                              int64
BLOCKID                        float64
SUMLEVEL                         int64
COUNTYID                         int64
STATEID                          int64
state                           object
state_ab                        object
city                            object
place                           object
type                            object
primary                         object
zip_code                         int64
area_code                        int64
lat                            float64
lng                            float64
ALand                          float64
AWater                           int64
pop                              int64
male_pop                         int64
female_pop                       int64
rent_mean                      float64
rent_median                    float64
rent_stdev                     float64
rent_sample_weight             float64
rent_samples                   float64
rent_gt_10               

In [8]:
train_data.columns

Index(['UID', 'BLOCKID', 'SUMLEVEL', 'COUNTYID', 'STATEID', 'state',
       'state_ab', 'city', 'place', 'type', 'primary', 'zip_code', 'area_code',
       'lat', 'lng', 'ALand', 'AWater', 'pop', 'male_pop', 'female_pop',
       'rent_mean', 'rent_median', 'rent_stdev', 'rent_sample_weight',
       'rent_samples', 'rent_gt_10', 'rent_gt_15', 'rent_gt_20', 'rent_gt_25',
       'rent_gt_30', 'rent_gt_35', 'rent_gt_40', 'rent_gt_50',
       'universe_samples', 'used_samples', 'hi_mean', 'hi_median', 'hi_stdev',
       'hi_sample_weight', 'hi_samples', 'family_mean', 'family_median',
       'family_stdev', 'family_sample_weight', 'family_samples',
       'hc_mortgage_mean', 'hc_mortgage_median', 'hc_mortgage_stdev',
       'hc_mortgage_sample_weight', 'hc_mortgage_samples', 'hc_mean',
       'hc_median', 'hc_stdev', 'hc_samples', 'hc_sample_weight',
       'home_equity_second_mortgage', 'second_mortgage', 'home_equity', 'debt',
       'second_mortgage_cdf', 'home_equity_cdf', 'debt_cdf', '

In [9]:
A=pd.DataFrame(train_data.columns)
A

,0
0,UID
1,BLOCKID
2,SUMLEVEL
3,COUNTYID
4,STATEID
5,state
6,state_ab
7,city
8,place
9,type


In [10]:
train_data.shape

(27321, 80)

In [11]:
train_data.isnull().sum()

UID                                0
BLOCKID                        27321
SUMLEVEL                           0
COUNTYID                           0
STATEID                            0
state                              0
state_ab                           0
city                               0
place                              0
type                               0
primary                            0
zip_code                           0
area_code                          0
lat                                0
lng                                0
ALand                              0
AWater                             0
pop                                0
male_pop                           0
female_pop                         0
rent_mean                        314
rent_median                      314
rent_stdev                       314
rent_sample_weight               314
rent_samples                     314
rent_gt_10                       314
rent_gt_15                       314
r

In [12]:
train_data.duplicated().sum()

160

1.There are 80 columns with 27321 records in the training data </br>
2.There are null values in the columns</br>
3.There are 160 duplicated records</br>
4.The data types range from int to float 

## Treating Missing Values

## Number of columns with missing values

In [13]:
len(train_data.columns[train_data.isnull().sum(axis=0 )> 0])

59

There are 59 columns with missing values.

## percentage of missings values in each column 

In [14]:
percent_missing = train_data.isnull().sum() * 100 / len(train_data)
missing_value_df = pd.DataFrame({'column_name': train_data.columns,
                                 'percent_missing': percent_missing})

In [15]:
missing_value_df

,column_name,percent_missing
UID,UID,0.000000
BLOCKID,BLOCKID,100.000000
SUMLEVEL,SUMLEVEL,0.000000
COUNTYID,COUNTYID,0.000000
STATEID,STATEID,0.000000
state,state,0.000000
state_ab,state_ab,0.000000
city,city,0.000000
place,place,0.000000
type,type,0.000000



We can delete the column "BLOCK ID" as it has all null values 

In [16]:
train_data.drop('BLOCKID', axis=1 , inplace=True)

## Calculate the overall percentage of missing data in the dataset

In [17]:
Missing_values=train_data[train_data.isnull().any(axis=1)]
Missing_values

,UID,SUMLEVEL,COUNTYID,STATEID,state,state_ab,city,place,type,primary,zip_code,area_code,lat,lng,ALand,AWater,pop,male_pop,female_pop,rent_mean,rent_median,rent_stdev,rent_sample_weight,rent_samples,rent_gt_10,rent_gt_15,rent_gt_20,rent_gt_25,rent_gt_30,rent_gt_35,rent_gt_40,rent_gt_50,universe_samples,used_samples,hi_mean,hi_median,hi_stdev,hi_sample_weight,hi_samples,family_mean,family_median,family_stdev,family_sample_weight,family_samples,hc_mortgage_mean,hc_mortgage_median,hc_mortgage_stdev,hc_mortgage_sample_weight,hc_mortgage_samples,hc_mean,hc_median,hc_stdev,hc_samples,hc_sample_weight,home_equity_second_mortgage,second_mortgage,home_equity,debt,second_mortgage_cdf,home_equity_cdf,debt_cdf,hs_degree,hs_degree_male,hs_degree_female,male_age_mean,male_age_median,male_age_stdev,male_age_sample_weight,male_age_samples,female_age_mean,female_age_median,female_age_stdev,female_age_sample_weight,female_age_samples,pct_own,married,married_snp,separated,divorced
51,223593,140,19,4,Arizona,AZ,Tucson,Littletown,CDP,tract,85734,520,32.067721,-110.867177,2909152.0,0,4531,4370,161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.64962,0.63734,0.91925,36.73330,34.50000,12.85240,1145.08910,4370.0,40.02370,40.83333,8.49563,30.01695,161.0,NaN,0.16308,0.16308,0.02634,0.20499
94,233040,140,101,8,Colorado,CO,Pueblo,Pueblo City,City,tract,81001,719,38.308407,-104.577687,1042529.0,0,579,270,309,782.00000,781.0,22.95830,4.00000,4.0,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,10,4,19787.76153,24356.0,8348.64517,10.00000,10.0,11749.65000,11870.0,2295.73336,4.00000,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,1.00000,1.00000,19.90717,19.66667,1.76353,180.13479,270.0,20.00784,19.25000,4.30291,172.56153,309.0,0.00000,0.00000,0.00000,0.00000,0.00000
153,263292,140,13,34,New Jersey,NJ,Newark,Silver Lake,City,tract,7107,973,40.773830,-74.175141,763973.0,22300,3458,1787,1671,890.69365,929.0,472.84907,566.54167,1011.0,0.96211,0.79163,0.72981,0.66899,0.49551,0.40279,0.38883,0.29412,1011,1003,46259.09568,28496.0,52060.62169,820.16516,1228.0,55882.00264,41924.0,45573.64511,477.68311,746.0,2342.64044,2190.0,770.94014,48.54139,217.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.28571,1.0,1.0,0.03005,0.0,0.70552,0.73876,0.67054,34.66877,34.41667,19.38107,411.29534,1787.0,35.47667,35.58333,20.62717,369.61740,1671.0,0.24002,0.37411,0.05579,0.02504,0.07654
302,267158,140,47,36,New York,NY,Brooklyn,New York City,City,tract,11215,718,40.659126,-73.969773,2313042.0,227326,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,292484,140,25,55,Wisconsin,WI,Madison,Madison City,City,tract,53703,608,43.073403,-89.395430,175748.0,0,3274,1293,1981,1191.78679,956.0,737.36689,768.38091,1497.0,1.00000,0.98039,0.96757,0.96757,0.96154,0.90422,0.84992,0.78733,1505,1326,14917.36079,9758.0,15534.75750,1202.89330,1505.0,22527.10680,7965.0,21141.68686,23.49730,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.87107,0.89950,0.82353,24.13033,22.08333,7.84307,755.14608,1293.0,22.03226,21.08333,5.13435,1365.86300,1981.0,0.00000,0.00773,0.00000,0.00000,0.01160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27127,266321,140,5,36,New York,NY,Bronx,Mount Vernon City,City,tract,10458,718,40.874095,-73.885197,173358.0,0,7561,3640,3921,1183.94615,1171.0,410.85034,877.82915,2618.0,0.97718,0.88820,0.72495,0.65377,0.55861,0.48240,0.43327,0.28936,2632,2585,561

In [18]:
Missing_values.shape

(736, 79)

In [19]:
train_data.shape

(27321, 79)

In [20]:
missing_percentage= (736/27321) * 100
missing_percentage

2.693898466381172

The overall missing data in the dataset is 2.69 which is negligable, therefore the missing values can be dropped.

## Dropping NULL/NAN values 

In [21]:
train_data.dropna()

,UID,SUMLEVEL,COUNTYID,STATEID,state,state_ab,city,place,type,primary,zip_code,area_code,lat,lng,ALand,AWater,pop,male_pop,female_pop,rent_mean,rent_median,rent_stdev,rent_sample_weight,rent_samples,rent_gt_10,rent_gt_15,rent_gt_20,rent_gt_25,rent_gt_30,rent_gt_35,rent_gt_40,rent_gt_50,universe_samples,used_samples,hi_mean,hi_median,hi_stdev,hi_sample_weight,hi_samples,family_mean,family_median,family_stdev,family_sample_weight,family_samples,hc_mortgage_mean,hc_mortgage_median,hc_mortgage_stdev,hc_mortgage_sample_weight,hc_mortgage_samples,hc_mean,hc_median,hc_stdev,hc_samples,hc_sample_weight,home_equity_second_mortgage,second_mortgage,home_equity,debt,second_mortgage_cdf,home_equity_cdf,debt_cdf,hs_degree,hs_degree_male,hs_degree_female,male_age_mean,male_age_median,male_age_stdev,male_age_sample_weight,male_age_samples,female_age_mean,female_age_median,female_age_stdev,female_age_sample_weight,female_age_samples,pct_own,married,married_snp,separated,divorced
0,267822,140,53,36,New York,NY,Hamilton,Hamilton,City,tract,13346,315,42.840812,-75.501524,2.021834e+08,1699120,5230,2612,2618,769.38638,784.0,232.63967,272.34441,362.0,0.86761,0.79155,0.59155,0.45634,0.42817,0.18592,0.15493,0.12958,387,355,63125.28406,48120.0,49042.01206,1290.96240,2024.0,67994.14790,53245.0,47667.30119,884.33516,1491.0,1414.80295,1223.0,641.22898,377.83135,867.0,570.01530,558.0,270.11299,770.0,499.29293,0.01588,0.02077,0.08919,0.52963,0.43658,0.49087,0.73341,0.89288,0.85880,0.92434,42.48574,44.00000,22.97306,696.42136,2612.0,44.48629,45.33333,22.51276,685.33845,2618.0,0.79046,0.57851,0.01882,0.01240,0.08770
1,246444,140,141,18,Indiana,IN,South Bend,Roseland,City,tract,46616,574,41.701441,-86.266614,1.560828e+06,100363,2633,1349,1284,804.87924,848.0,253.46747,312.58622,513.0,0.97410,0.93227,0.69920,0.69920,0.55179,0.41235,0.39044,0.27888,542,502,41931.92593,35186.0,31639.50203,838.74664,1127.0,50670.10337,43023.0,34715.57548,375.28798,554.0,864.41390,784.0,482.27020,316.88320,356.0,351.98293,336.0,125.40457,229.0,189.60606,0.02222,0.02222,0.04274,0.60855,0.42174,0.70823,0.58120,0.90487,0.86947,0.94187,34.84728,32.00000,20.37452,323.90204,1349.0,36.48391,37.58333,23.43353,267.23367,1284.0,0.52483,0.34886,0.01426,0.01426,0.09030
2,245683,140,63,18,Indiana,IN,Danville,Danville,City,tract,46122,317,39.792202,-86.515246,6.956160e+07,284193,6881,3643,3238,742.77365,703.0,323.39011,291.85520,378.0,0.95238,0.88624,0.79630,0.66667,0.39153,0.39153,0.28307,0.15873,459,378,84942.68317,74964.0,56811.62186,1155.20980,2488.0,95262.51431,85395.0,49292.67664,709.74925,1889.0,1506.06758,1361.0,731.89394,699.41354,1491.0,556.45986,532.0,184.42175,538.0,323.35354,0.00000,0.00000,0.09512,0.73484,1.00000,0.46332,0.28704,0.94288,0.94616,0.93952,39.38154,40.83333,22.89769,888.29730,3643.0,42.15810,42.83333,23.94119,707.01963,3238.0,0.85331,0.64745,0.02830,0.01607,0.10657
3,279653,140,127,72,Puerto Rico,PR,San Juan,Guaynabo,Urban,tract,927,787,18.396103,-66.104169,1.105793e+06,0,2700,1141,1559,803.42018,782.0,297.39258,259.30316,368.0,0.94693,0.87151,0.69832,0.61732,0.51397,0.46927,0.35754,0.32961,438,358,48733.67116,37845.0,45100.54010,928.32193,1267.0,56401.68133,44399.0,41082.90515,490.18479,729.0,1175.28642,1101.0,428.98751,261.28471,437.0,288.04047,247.0,185.55887,392.0,314.90566,0.01086,0.01086,0.01086,0.52714,0.53057,0.82530,0.73727,0.91500,0.90755,0.92043,48.64749,48.91667,23.05968,274.98956,1141.0,47.77526,50.58333,24.32015,362.20193,1559.0,0.65037,0.47257,0.02021,0.02021,0.10106
4,247218,140,161,20,Kansas,KS,Manhattan,Manhattan City,City,tract,66502,785,39.195573,-96.569366,2.554403e+06,0,5637,2586,3051,938.56493,881.0,392.44096,1005.42886,1704.0,0.99286,0.98247,0.91688,0.84740,0.78247,0.60974,0.55455,0.44416,1725,1540,31834.15466,22497.0,34046.50907,1548.67477,1983.0,54053.42396,50272.0,39609.12605,244.08903,395.0,1192.58759,1125.0,327.49674,76.61052,134.0,443.68855,444.0,76.12674,124.0,79.55556,0.05426,0.05426,0.05426,0.51938,0.18332,0.65545,0.74967,1.00000,1.0000

In [22]:
train_data.duplicated().sum()

160

In [23]:
train_data.drop_duplicates(inplace=True)

In [24]:
train_data.duplicated().sum()

0

## Final dataset size

In [25]:
train_data.shape

(27161, 79)

## Same will be performed on the test data

In [26]:
test_data.isnull().sum()

UID                                0
BLOCKID                        11709
SUMLEVEL                           0
COUNTYID                           0
STATEID                            0
state                              0
state_ab                           0
city                               0
place                              0
type                               0
primary                            0
zip_code                           0
area_code                          0
lat                                0
lng                                0
ALand                              0
AWater                             0
pop                                0
male_pop                           0
female_pop                         0
rent_mean                        148
rent_median                      148
rent_stdev                       148
rent_sample_weight               148
rent_samples                     148
rent_gt_10                       149
rent_gt_15                       149
r

In [27]:
percent_missing1 = test_data.isnull().sum() * 100 / len(test_data)
missing_value_df1 = pd.DataFrame({'column_name': test_data.columns,
                                 'percent_missing': percent_missing})

In [28]:
missing_value_df1

,column_name,percent_missing
UID,UID,0.000000
BLOCKID,BLOCKID,100.000000
SUMLEVEL,SUMLEVEL,0.000000
COUNTYID,COUNTYID,0.000000
STATEID,STATEID,0.000000
state,state,0.000000
state_ab,state_ab,0.000000
city,city,0.000000
place,place,0.000000
type,type,0.000000


In [29]:
test_data.drop('BLOCKID', axis=1 , inplace=True)

In [30]:
Missing_values1=test_data[test_data.isnull().any(axis=1)]
Missing_values1

,UID,SUMLEVEL,COUNTYID,STATEID,state,state_ab,city,place,type,primary,zip_code,area_code,lat,lng,ALand,AWater,pop,male_pop,female_pop,rent_mean,rent_median,rent_stdev,rent_sample_weight,rent_samples,rent_gt_10,rent_gt_15,rent_gt_20,rent_gt_25,rent_gt_30,rent_gt_35,rent_gt_40,rent_gt_50,universe_samples,used_samples,hi_mean,hi_median,hi_stdev,hi_sample_weight,hi_samples,family_mean,family_median,family_stdev,family_sample_weight,family_samples,hc_mortgage_mean,hc_mortgage_median,hc_mortgage_stdev,hc_mortgage_sample_weight,hc_mortgage_samples,hc_mean,hc_median,hc_stdev,hc_samples,hc_sample_weight,home_equity_second_mortgage,second_mortgage,home_equity,debt,second_mortgage_cdf,home_equity_cdf,debt_cdf,hs_degree,hs_degree_male,hs_degree_female,male_age_mean,male_age_median,male_age_stdev,male_age_sample_weight,male_age_samples,female_age_mean,female_age_median,female_age_stdev,female_age_sample_weight,female_age_samples,pct_own,married,married_snp,separated,divorced
17,265339,140,3,32,Nevada,NV,Las Vegas,Winchester,City,tract,89119,702,36.111448,-115.159125,612803,0,724,490,234,751.50703,736.0,172.34699,446.71620,501.0,0.95781,0.91772,0.87764,0.77637,0.57806,0.45992,0.43460,0.29325,501,474,32947.89919,24961.0,37213.92684,434.21963,501.0,27064.98901,27879.0,16391.07515,58.49710,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.89028,0.88703,0.90000,47.99025,46.58333,14.22954,108.69207,490.0,33.57247,32.50000,17.36519,49.31407,234.0,0.00000,0.22857,0.11020,0.06122,0.26327
27,287596,140,451,48,Texas,TX,San Angelo,San Angelo City,Town,tract,76903,325,31.431831,-100.404539,4603626,0,3099,2231,868,1061.03134,1079.0,221.25258,32.20117,91.0,1.00000,1.00000,1.00000,0.89011,0.59341,0.48352,0.18681,0.08791,117,91,42867.67343,40385.0,16658.79976,100.23177,117.0,44652.33710,41215.0,15428.79215,93.56564,109.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.97806,0.99194,0.92958,22.78803,21.00000,5.89881,1140.80210,2231.0,21.40298,20.50000,7.28394,456.32778,868.0,0.00000,0.22232,0.17475,0.01052,0.00000
44,250903,140,25,25,Massachusetts,MA,Cambridge,Cambridge City,City,tract,2139,617,42.359478,-71.121263,1158117,0,6234,2518,3716,1964.19623,2025.0,769.92869,137.01400,646.0,1.00000,0.98262,0.90521,0.87204,0.77725,0.68562,0.60506,0.52607,646,633,54583.21233,33796.0,53650.39665,427.80531,661.0,72208.68011,70655.0,51468.81482,75.59543,158.0,1943.06061,1911.0,265.85378,2.97595,15.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.33333,1.0,1.0,0.01506,0.0,0.94958,0.93675,0.96093,23.68423,21.33333,9.46337,1279.90666,2518.0,22.53871,20.75000,7.40442,2069.57453,3716.0,0.02169,0.10879,0.05440,0.00204,0.00409
54,287557,140,441,48,Texas,TX,Abilene,Tye City,Town,tract,79607,325,32.423876,-99.842475,18425872,194540,916,857,59,972.00000,968.0,50.76498,2.96970,6.0,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,6,6,18277.16343,9778.0,13912.23552,8.99920,11.0,32499.50000,32500.0,1825.74186,5.00000,5.0,NaN,NaN,NaN,NaN,NaN,224.5,224.0,18.25742,5.0,5.00000,0.0,0.0,0.00000,0.0,1.0,1.00000,1.0,0.95455,0.95455,NaN,22.47792,21.58333,5.10998,597.48392,857.0,22.72458,23.16667,2.18207,26.71180,59.0,0.60000,0.01984,0.00933,0.00000,0.02217
70,247510,140,209,20,Kansas,KS,Kansas City,Kansas City City,City,tract,66104,913,39.171767,-94.680617,8005557,1678971,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11543,267675,140,47,36,New York,NY,Brooklyn,New York City,City,tract,11212,718,40.670130,-73.908859,195831,0,4318,1772,2546,610

In [31]:
test_data.shape

(11709, 79)

In [32]:
missing_percentage= (354/11709) * 100
missing_percentage

3.0233153984114787

In [33]:
test_data.dropna()

,UID,SUMLEVEL,COUNTYID,STATEID,state,state_ab,city,place,type,primary,zip_code,area_code,lat,lng,ALand,AWater,pop,male_pop,female_pop,rent_mean,rent_median,rent_stdev,rent_sample_weight,rent_samples,rent_gt_10,rent_gt_15,rent_gt_20,rent_gt_25,rent_gt_30,rent_gt_35,rent_gt_40,rent_gt_50,universe_samples,used_samples,hi_mean,hi_median,hi_stdev,hi_sample_weight,hi_samples,family_mean,family_median,family_stdev,family_sample_weight,family_samples,hc_mortgage_mean,hc_mortgage_median,hc_mortgage_stdev,hc_mortgage_sample_weight,hc_mortgage_samples,hc_mean,hc_median,hc_stdev,hc_samples,hc_sample_weight,home_equity_second_mortgage,second_mortgage,home_equity,debt,second_mortgage_cdf,home_equity_cdf,debt_cdf,hs_degree,hs_degree_male,hs_degree_female,male_age_mean,male_age_median,male_age_stdev,male_age_sample_weight,male_age_samples,female_age_mean,female_age_median,female_age_stdev,female_age_sample_weight,female_age_samples,pct_own,married,married_snp,separated,divorced
0,255504,140,163,26,Michigan,MI,Detroit,Dearborn Heights City,CDP,tract,48239,313,42.346422,-83.252823,2711280,39555,3417,1479,1938,858.57169,859.0,232.39082,276.07497,424.0,1.00000,0.95696,0.85316,0.85316,0.85316,0.85316,0.76962,0.63544,435,395,48899.52121,38746.0,44392.20902,798.02401,1180.0,53802.87122,45167.0,43756.56479,464.30972,769.0,1139.24548,1109.0,336.47710,262.67011,474.0,488.51323,436.0,192.75147,271.0,189.18182,0.06443,0.06443,0.07651,0.63624,0.14111,0.55087,0.51965,0.91047,0.92010,0.90391,33.37131,27.83333,22.36768,334.30978,1479.0,34.78682,33.75000,21.58531,416.48097,1938.0,0.70252,0.28217,0.05910,0.03813,0.14299
1,252676,140,1,23,Maine,ME,Auburn,Auburn City,City,tract,4210,207,44.100724,-70.257832,14778785,2705204,3796,1846,1950,832.68625,750.0,267.22342,183.32299,245.0,1.00000,1.00000,0.86611,0.67364,0.30962,0.30962,0.30962,0.27197,275,239,72335.33234,61008.0,51895.81159,922.82969,1722.0,85642.22095,74759.0,49156.72870,482.99945,1147.0,1533.25988,1438.0,536.61118,373.96188,937.0,661.31296,668.0,201.31365,510.0,279.69697,0.01175,0.01175,0.14375,0.64755,0.52310,0.26442,0.49359,0.94290,0.92832,0.95736,43.88680,46.08333,22.90302,427.10824,1846.0,44.23451,46.66667,22.37036,532.03505,1950.0,0.85128,0.64221,0.02338,0.00000,0.13377
2,276314,140,15,42,Pennsylvania,PA,Pine City,Millerton,Borough,tract,14871,607,41.948556,-76.783808,258903666,863840,3944,2065,1879,816.00639,755.0,416.25699,141.39063,217.0,0.97573,0.93204,0.78641,0.71845,0.63592,0.47573,0.43689,0.32524,245,206,58501.15901,51648.0,45245.27248,893.07759,1461.0,65694.06582,57186.0,44239.31893,619.73962,1084.0,1254.54462,1089.0,596.85204,340.45884,552.0,397.44466,356.0,189.40372,664.0,534.16737,0.01069,0.01316,0.06497,0.45395,0.51066,0.60484,0.83848,0.89238,0.86003,0.92463,39.81661,41.91667,24.29111,499.10080,2065.0,41.62426,44.50000,22.86213,453.11959,1879.0,0.81897,0.59961,0.01746,0.01358,0.10026
3,248614,140,231,21,Kentucky,KY,Monticello,Monticello City,City,tract,42633,606,36.746009,-84.766870,501694825,2623067,2508,1427,1081,418.68937,385.0,156.92024,88.95960,93.0,1.00000,0.93548,0.93548,0.64516,0.55914,0.46237,0.46237,0.36559,153,93,38237.55059,31612.0,34527.61607,775.17947,957.0,44156.38709,34687.0,34899.74300,535.21987,689.0,862.65763,749.0,624.42157,299.56752,337.0,200.88113,180.0,91.56490,467.0,454.85404,0.00995,0.00995,0.01741,0.41915,0.53770,0.80931,0.87403,0.60908,0.56584,0.65947,41.81638,43.00000,24.65325,333.57733,1427.0,44.81200,48.00000,21.03155,263.94320,1081.0,0.84609,0.56953,0.05492,0.04694,0.12489
4,286865,140,355,48,Texas,TX,Corpus Christi,Edroy,Town,tract,78410,361,27.882461,-97.678586,13796057,497689,6230,3274,2956,1031.63763,997.0,326.76727,277.39844,624.0,0.72276,0.66506,0.53526,0.38301,0.18910,0.16667,0.14263,0.11058,660,624,114456.07790,94211.0,81950.95692,836.30759,2404.0,123527.02420,103898.0,72173.55823,507.42257,1738.0,1996.41425,1907.0,740.21168,319.97570,1102.0,867.57713,804.0,376.20236,642.0,333.91919,0.00000,0.00000,0.03440,0.63188,1.00000,0.74519,0.52943,0.86297,0

In [34]:
test_data.drop_duplicates(inplace=True)

In [35]:
test_data.duplicated().sum()

0

## TEST DATA FINAL SIZE 

In [36]:
test_data.shape

(11677, 79)

### Perform EDA and come out with insights into population density and age. You may require deriving new fields (Make sure to weight averages for accurate measurements): 
### •	Population density (hint-use ‘pop’ and ‘Aland’ to calculate)</br>
### •	median age (hint-use the variables ‘male_age_median’, ‘female_age_median’, ‘male_pop’, ‘female_pop’)</br>
### Visualize the findings using appropriate chart type.


In [37]:
df= train_data[['state', 'city', 'place','type','ALand', 'pop', 'male_age_median', 'female_age_median', 'male_pop', 'female_pop']]
df.head()

,state,city,place,type,ALand,pop,male_age_median,female_age_median,male_pop,female_pop
0,New York,Hamilton,Hamilton,City,202183361.0,5230,44.00000,45.33333,2612,2618
1,Indiana,South Bend,Roseland,City,1560828.0,2633,32.00000,37.58333,1349,1284
2,Indiana,Danville,Danville,City,69561595.0,6881,40.83333,42.83333,3643,3238
3,Puerto Rico,San Juan,Guaynabo,Urban,1105793.0,2700,48.91667,50.58333,1141,1559
4,Kansas,Manhattan,Manhattan City,City,2554403.0,5637,22.41667,21.58333,2586,3051


In [38]:
df.shape

(27161, 10)

## Population density is total population by total area of land; Therefore we will use total population as pop and total area of land as ALand 

In [39]:
pd.options.mode.chained_assignment = None 

In [40]:
df['p_density'] = df['pop'] / df['ALand']

## Median age will be average of of median age of male and median age of female.

In [45]:
df['median_age'] = (df['male_age_median']+ df['female_age_median']) / 2

In [46]:
df.head()

,state,city,place,type,ALand,pop,male_age_median,female_age_median,male_pop,female_pop,p_density,median_age
0,New York,Hamilton,Hamilton,City,202183361.0,5230,44.00000,45.33333,2612,2618,0.000026,44.666665
1,Indiana,South Bend,Roseland,City,1560828.0,2633,32.00000,37.58333,1349,1284,0.001687,34.791665
2,Indiana,Danville,Danville,City,69561595.0,6881,40.83333,42.83333,3643,3238,0.000099,41.833330
3,Puerto Rico,San Juan,Guaynabo,Urban,1105793.0,2700,48.91667,50.58333,1141,1559,0.002442,49.750000
4,Kansas,Manhattan,Manhattan City,City,2554403.0,5637,22.41667,21.58333,2586,3051,0.002207,22.000000


Load the df dataset in the excel sheet to perform visualisations.

In [49]:
df.to_excel('A.xlsx')

In [69]:
#--> USED POWERBI For visualisations


### Create bins for population into a new variable by selecting appropriate class interval so that the no of categories(bins) don’t exceed 5 for the ease of analysis. Analyze the married, separated and divorced population for these population brackets. Visualize using appropriate chart type.


In [52]:
train_data.head()

,UID,SUMLEVEL,COUNTYID,STATEID,state,state_ab,city,place,type,primary,zip_code,area_code,lat,lng,ALand,AWater,pop,male_pop,female_pop,rent_mean,rent_median,rent_stdev,rent_sample_weight,rent_samples,rent_gt_10,rent_gt_15,rent_gt_20,rent_gt_25,rent_gt_30,rent_gt_35,rent_gt_40,rent_gt_50,universe_samples,used_samples,hi_mean,hi_median,hi_stdev,hi_sample_weight,hi_samples,family_mean,family_median,family_stdev,family_sample_weight,family_samples,hc_mortgage_mean,hc_mortgage_median,hc_mortgage_stdev,hc_mortgage_sample_weight,hc_mortgage_samples,hc_mean,hc_median,hc_stdev,hc_samples,hc_sample_weight,home_equity_second_mortgage,second_mortgage,home_equity,debt,second_mortgage_cdf,home_equity_cdf,debt_cdf,hs_degree,hs_degree_male,hs_degree_female,male_age_mean,male_age_median,male_age_stdev,male_age_sample_weight,male_age_samples,female_age_mean,female_age_median,female_age_stdev,female_age_sample_weight,female_age_samples,pct_own,married,married_snp,separated,divorced
0,267822,140,53,36,New York,NY,Hamilton,Hamilton,City,tract,13346,315,42.840812,-75.501524,202183361.0,1699120,5230,2612,2618,769.38638,784.0,232.63967,272.34441,362.0,0.86761,0.79155,0.59155,0.45634,0.42817,0.18592,0.15493,0.12958,387,355,63125.28406,48120.0,49042.01206,1290.96240,2024.0,67994.14790,53245.0,47667.30119,884.33516,1491.0,1414.80295,1223.0,641.22898,377.83135,867.0,570.01530,558.0,270.11299,770.0,499.29293,0.01588,0.02077,0.08919,0.52963,0.43658,0.49087,0.73341,0.89288,0.85880,0.92434,42.48574,44.00000,22.97306,696.42136,2612.0,44.48629,45.33333,22.51276,685.33845,2618.0,0.79046,0.57851,0.01882,0.01240,0.08770
1,246444,140,141,18,Indiana,IN,South Bend,Roseland,City,tract,46616,574,41.701441,-86.266614,1560828.0,100363,2633,1349,1284,804.87924,848.0,253.46747,312.58622,513.0,0.97410,0.93227,0.69920,0.69920,0.55179,0.41235,0.39044,0.27888,542,502,41931.92593,35186.0,31639.50203,838.74664,1127.0,50670.10337,43023.0,34715.57548,375.28798,554.0,864.41390,784.0,482.27020,316.88320,356.0,351.98293,336.0,125.40457,229.0,189.60606,0.02222,0.02222,0.04274,0.60855,0.42174,0.70823,0.58120,0.90487,0.86947,0.94187,34.84728,32.00000,20.37452,323.90204,1349.0,36.48391,37.58333,23.43353,267.23367,1284.0,0.52483,0.34886,0.01426,0.01426,0.09030
2,245683,140,63,18,Indiana,IN,Danville,Danville,City,tract,46122,317,39.792202,-86.515246,69561595.0,284193,6881,3643,3238,742.77365,703.0,323.39011,291.85520,378.0,0.95238,0.88624,0.79630,0.66667,0.39153,0.39153,0.28307,0.15873,459,378,84942.68317,74964.0,56811.62186,1155.20980,2488.0,95262.51431,85395.0,49292.67664,709.74925,1889.0,1506.06758,1361.0,731.89394,699.41354,1491.0,556.45986,532.0,184.42175,538.0,323.35354,0.00000,0.00000,0.09512,0.73484,1.00000,0.46332,0.28704,0.94288,0.94616,0.93952,39.38154,40.83333,22.89769,888.29730,3643.0,42.15810,42.83333,23.94119,707.01963,3238.0,0.85331,0.64745,0.02830,0.01607,0.10657
3,279653,140,127,72,Puerto Rico,PR,San Juan,Guaynabo,Urban,tract,927,787,18.396103,-66.104169,1105793.0,0,2700,1141,1559,803.42018,782.0,297.39258,259.30316,368.0,0.94693,0.87151,0.69832,0.61732,0.51397,0.46927,0.35754,0.32961,438,358,48733.67116,37845.0,45100.54010,928.32193,1267.0,56401.68133,44399.0,41082.90515,490.18479,729.0,1175.28642,1101.0,428.98751,261.28471,437.0,288.04047,247.0,185.55887,392.0,314.90566,0.01086,0.01086,0.01086,0.52714,0.53057,0.82530,0.73727,0.91500,0.90755,0.92043,48.64749,48.91667,23.05968,274.98956,1141.0,47.77526,50.58333,24.32015,362.20193,1559.0,0.65037,0.47257,0.02021,0.02021,0.10106
4,247218,140,161,20,Kansas,KS,Manhattan,Manhattan City,City,tract,66502,785,39.195573,-96.569366,2554403.0,0,5637,2586,3051,938.56493,881.0,392.44096,1005.42886,1704.0,0.99286,0.98247,0.91688,0.84740,0.78247,0.60974,0.55455,0.44416,1725,1540,31834.15466,22497.0,34046.50907,1548.67477,1983.0,54053.42396,50272.0,39609.12605,244.08903,395.0,1192.58759,1125.0,327.49674,76.61052,134.0,443.68855,444.0,76.12674,124.0,79.55556,0.05426,0.05426,0.05426,0.51938,0.18332,0.65545,0.74967,1.00000,1.00000,1.00000,26

## lets find the maximum value of male mean age and female mean age in the dataset.

In [55]:
train_data["male_age_mean"].max()

77.75992

In [57]:
train_data["female_age_mean"].max()

79.83739

## lets find the minimum value of male mean age and female mean age in the dataset.

In [61]:
train_data["male_age_mean"].min()

12.14583

In [59]:
train_data["female_age_mean"].min()

16.008329999999997

In [62]:
age_df = train_data[['state', 'city', 'place', 'pop', 'male_pop', 'female_pop', 'male_age_median', 'female_age_median', 'married', 'separated', 'divorced']]

## THE BRACKETS :

YOUNG YOUTH - <18 </br>
YOUTH - 19 to 30  </br>
ADULT - 30 to 55  </br>
SENIOR - >55      </br>

In [64]:
bins = [0,18, 30, 55, 100]
labels = ['YOUNG YOUTH', 'YOUTH', ' ADULLT' , 'SENIOR']

In [65]:
age_df['male_population_bracket'] = pd.cut(age_df['male_age_median'], bins, labels = labels)

In [66]:

age_df['female_population_bracket'] = pd.cut(age_df['female_age_median'], bins, labels = labels)

In [67]:
age_df.head()


,state,city,place,pop,male_pop,female_pop,male_age_median,female_age_median,married,separated,divorced,male_population_bracket,female_population_bracket
0,New York,Hamilton,Hamilton,5230,2612,2618,44.00000,45.33333,0.57851,0.01240,0.08770,Adult,Adult
1,Indiana,South Bend,Roseland,2633,1349,1284,32.00000,37.58333,0.34886,0.01426,0.09030,Adult,Adult
2,Indiana,Danville,Danville,6881,3643,3238,40.83333,42.83333,0.64745,0.01607,0.10657,Adult,Adult
3,Puerto Rico,San Juan,Guaynabo,2700,1141,1559,48.91667,50.58333,0.47257,0.02021,0.10106,Adult,Adult
4,Kansas,Manhattan,Manhattan City,5637,2586,3051,22.41667,21.58333,0.12356,0.00000,0.03109,YOUTH,YOUTH


Load the df dataset in the excel sheet to perform visualisations.

In [71]:
age_df.to_excel('B.xlsx')

In [70]:
#--> USED POWERBI For visualisations